In [1]:
from pathlib import Path
from validphys.loader import _get_nnpdf_profile
from validphys.api import API
import numpy as np
import pandas as pd
from validphys.convolution import central_predictions

profile = _get_nnpdf_profile()
yaml_db = Path(profile["data_path"]) / "yamldb"

The `yaml_db` folder is a temporary thing as it contains files that look like:

```yaml
conversion_factor: 1.0
operands:
- - NMC_NC_EM_D_F2
- - NMC_NC_EM_P_F2
operation: RATIO
target_dataset: NMCPD
```

This information will eventually be part of the new commondata format of course.

The `operation` is applied to the first level of the list while the second level is just concatenated. This is necessary since `pineappl` fktables might contain one layer of concatenation which is already done for the "classic" fktables.

The `pineappl` fktables will live inside the appropiate `theory_xxx` folder `/pineappls`.

In [2]:
# Test them all
if True:
    from yaml import safe_load
    pdf = API.pdf(pdf="NNPDF40_nnlo_as_01180")
    all_res = []
    nnpdf40_runcard = safe_load(Path("/home/juacrumar/NNPDF-testing/nnpdf/n3fit/NNPDF40_with_pineappl.yml").read_text())
    #nnpdf40_runcard = safe_load(Path("/mount/storage/Academic_Workspace/NNPDF/source/nnpdf/n3fit/NNPDF40_with_pineappl.yml").read_text())
    for d in nnpdf40_runcard["dataset_inputs"]:
        target_ds = d["dataset"]
        cfac = d.get("cfac", [])
        old_ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac + ["oldmode"]}, theoryid=200, use_cuts="internal")
        ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac}, theoryid=200, use_cuts="internal")
        new_cp = central_predictions(ds, pdf)
        cp = central_predictions(old_ds, pdf)
        all_res.append(pd.concat([new_cp, cp, new_cp/cp], axis=1, keys=["vp", "pine", f"ratio {target_ds}, {cfac}"]))
        
    for i in all_res:
        mean_ratio = i[i.columns[2]].mean()
        if not (0.9 < mean_ratio < 1.1):
            print(i)


-- Reading COMMONDATA for Dataset: NMCPD_dw_ite
nData: 260 nSys: 105
-- COMMONDATA Files for NMCPD_dw_ite successfully read.

LHAPDF 6.4.0 loading /usr/share/lhapdf/LHAPDF/NNPDF40_nnlo_as_01180/NNPDF40_nnlo_as_01180_0000.dat
NNPDF40_nnlo_as_01180 PDF set, member #0, version 1

-- Reading COMMONDATA for Dataset: NMC
nData: 292 nSys: 16
-- COMMONDATA Files for NMC successfully read.


-- Reading COMMONDATA for Dataset: SLACP_dwsh
nData: 211 nSys: 3
-- COMMONDATA Files for SLACP_dwsh successfully read.


-- Reading COMMONDATA for Dataset: SLACD_dw_ite
nData: 211 nSys: 103
-- COMMONDATA Files for SLACD_dw_ite successfully read.


-- Reading COMMONDATA for Dataset: BCDMSP_dwsh
nData: 351 nSys: 11
-- COMMONDATA Files for BCDMSP_dwsh successfully read.


-- Reading COMMONDATA for Dataset: BCDMSD_dw_ite
nData: 254 nSys: 108
-- COMMONDATA Files for BCDMSD_dw_ite successfully read.


-- Reading COMMONDATA for Dataset: CHORUSNUPb_dw_ite
nData: 607 nSys: 1014
-- COMMONDATA Files for CHORUSNUPb_dw


-- Reading COMMONDATA for Dataset: CMS_SINGLETOP_TCH_TOT_7TEV
nData: 1 nSys: 3
-- COMMONDATA Files for CMS_SINGLETOP_TCH_TOT_7TEV successfully read.


-- Reading COMMONDATA for Dataset: CMS_SINGLETOP_TCH_R_8TEV
nData: 1 nSys: 1
-- COMMONDATA Files for CMS_SINGLETOP_TCH_R_8TEV successfully read.


-- Reading COMMONDATA for Dataset: CMS_SINGLETOP_TCH_R_13TEV
nData: 1 nSys: 1
-- COMMONDATA Files for CMS_SINGLETOP_TCH_R_13TEV successfully read.


-- Reading COMMONDATA for Dataset: LHCBZ940PB
nData: 9 nSys: 11
-- COMMONDATA Files for LHCBZ940PB successfully read.


-- Reading COMMONDATA for Dataset: LHCBZEE2FB_40
nData: 17 nSys: 19
-- COMMONDATA Files for LHCBZEE2FB_40 successfully read.


-- Reading COMMONDATA for Dataset: LHCBWZMU7TEV
nData: 33 nSys: 35
-- COMMONDATA Files for LHCBWZMU7TEV successfully read.


-- Reading COMMONDATA for Dataset: LHCBWZMU8TEV
nData: 34 nSys: 36
-- COMMONDATA Files for LHCBWZMU8TEV successfully read.


-- Reading COMMONDATA for Dataset: LHCB_Z_13TEV_DIMUON


In [13]:
target_ds = "CDFZRAP_NEW"
cfac = [] # ["NRM"]
old_ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac + ["oldmode"]}, theoryid=200, use_cuts="internal")
ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac}, theoryid=200, use_cuts="internal")

In [14]:
# Let's try to get a prediction out of it
pdf = API.pdf(pdf="NNPDF40_nnlo_as_01180")
new_cp = central_predictions(ds, pdf)
cp = central_predictions(old_ds, pdf)
pd.concat([new_cp, cp, cp/new_cp, new_cp/cp], axis=1, keys=["pine", "vp", "ratio vp/ratio", "ratio pine/vp"])

,pine,vp,ratio vp/ratio,ratio pine/vp
,0,0,0,0
data,,,,
0,69.494559,69.483791,0.999845,1.000155
1,69.268891,69.257658,0.999838,1.000162
2,68.694395,68.685647,0.999873,1.000127
3,68.775831,68.765783,0.999854,1.000146
4,67.924732,67.917252,0.999890,1.000110
5,67.688393,67.683064,0.999921,1.000079
6,66.878242,66.874985,0.999951,1.000049
7,65.647578,65.647331,0.999996,1.000004


In [5]:
pine_fkspec = ds.fkspecs[0]
old_fkspec = old_ds.fkspecs[0]

In [6]:
import pineappl
pines = [pineappl.fk_table.FkTable.read(i.as_posix()) for i in pine_fkspec.fkpath]
# Inspect the pineappl prediction
res_pine = []
pp = pines[0]
lpdf = pdf.load()

for p in pines:
    res_pine.append(p.convolute_with_one(2212, lpdf.central_member.xfxQ2))
total_pine = np.concatenate(res_pine)

LHAPDF 6.4.0 loading all 101 PDFs in set NNPDF40_nnlo_as_01180
NNPDF40_nnlo_as_01180, version 1; 101 PDF members


In [7]:
# Let's inspect the content of the old fktables, remove the cfactor for now
from validphys.fkparser import load_fktable
old_fkspec.cfactors = False
old_fktabledata = load_fktable(old_fkspec)

In [8]:
print(f"hadronic?: {old_fktabledata.hadronic}")
print(f"Q: {old_fktabledata.Q0}")
print(f"n: {old_fktabledata.ndata}")
print(f"xgrid shape: {old_fktabledata.xgrid.shape}")
#old_fktabledata.sigma

hadronic?: True
Q: 1.65
n: 33
xgrid shape: (40,)


In [9]:
# First read the metadata that vp `FKTableData` needs and that all subgrids share
Q0 = np.sqrt(pp.muf2())
xgrid = pp.x_grid()
# Hadronic means in practice that not all luminosity combinations are just electron X proton
hadronic = not all(-11 in i for i in pp.lumi())
# Now prepare the concatenation of grids
fktables = []
for p in pines:
    tmp = p.table().T/p.bin_normalizations()
    fktables.append(tmp.T)
fktable = np.concatenate(fktables, axis=0)
ndata = fktable.shape[0]

In [10]:
# Now let's try to join the fktable, luminosity and xgrid into a pandas dataframe
# keeping compatibility with validphys and, hopefully, 50% of my own sanity

# Step 1), make the luminosity into a 14x14 mask for the evolution basis
eko_numbering_scheme = (22, 100, 21, 200, 203, 208, 215, 224, 235, 103, 108, 115, 124, 135)
# note that this is the same ordering that was used in fktables
flavour_map = np.zeros((14, 14), dtype=bool)
for i, j in pp.lumi():
    idx = eko_numbering_scheme.index(i)
    jdx = eko_numbering_scheme.index(j)
    flavour_map[idx,jdx] = True
    
# Step 2) prepare the indices for the dataframe
xi = np.arange(len(xgrid))
ni = np.arange(ndata)
mi = pd.MultiIndex.from_product([ni, xi, xi], names=["data", "x1", "x2"])
co = np.where(flavour_map.ravel())[0]

# Step 3) Now play with the array until we flatten it in the right way?
# The fktables for pineappl have this extra factor of x...
# The output of pineappl is (ndata, flavours, x, x)
lf = len(co)
xfktable = fktable.reshape(ndata, lf, -1)/(xgrid[:,None]*xgrid[None,:]).flatten()
fkmod = np.moveaxis(xfktable, 1, -1)
fkframe = fkmod.reshape(-1, lf)

# Uff, big
df = pd.DataFrame(fkframe, index=mi, columns=co)

from validphys.convolution import central_hadron_predictions
from validphys.coredata import FKTableData
fk = FKTableData(sigma=df, ndata=ndata,  Q0=Q0, metadata=None, hadronic=True, xgrid=xgrid)
central_hadron_predictions(fk, pdf)

,0
data,
0,7.277419
1,21.190979
2,33.659136
3,44.967471
4,54.149089
5,60.857994
6,64.741567
7,65.445100
8,62.884963


In [11]:
# Create a luminosity tensor and check that the results are correct
from validphys.pdfbases import evolution

evol_basis = (
    "photon",
    "singlet",
    "g",
    "V",
    "V3",
    "V8",
    "V15",
    "V24",
    "V35",
    "T3",
    "T8",
    "T15",
    "T24",
    "T35",
)
total_pdf = evolution.grid_values(pdf, evol_basis, xgrid, [Q0]).squeeze()[0]/xgrid
print(total_pdf.shape)
lumi = np.einsum('ij,kl->ikjl', total_pdf, total_pdf)
lumi_masked = lumi[flavour_map]
print(fktable.shape)
print(lumi_masked.shape)
res = np.einsum('ijkl,jkl->i', fktable, lumi_masked)
#pd.concat([pd.DataFrame(res), cp, pd.DataFrame(res)/cp,  ], axis=1)

(14, 100)
(33, 91, 100, 100)
(91, 100, 100)


In [12]:
xfktable.reshape(48,91,-1).shape

(48, 91, 6875)

In [13]:
from validphys.fkparser import open_fkpath, _parse_string, _parse_header, _build_sigma
from validphys.fkparser import _parse_flavour_map, _parse_hadronic_fast_kernel
try:
    f.close()
except:
    pass
f = open_fkpath(old_fkspec.fkpath)
line_and_stream = enumerate(f, start=1)
lineno, header = next(line_and_stream)
res = {}
while True:
    marker, header_name = _parse_header(lineno, header)
    if header_name == "FastKernel":
        break
    if header_name == "FlavourMap":
        out, lineno, header = _parse_flavour_map(line_and_stream)
    else:
        out, lineno, header = _parse_string(line_and_stream)
    res[header_name] = out   

In [14]:
res["FlavourMap"].shape

(14, 14)

In [15]:
i_hate_pandas = _parse_hadronic_fast_kernel(f)

In [16]:
i_hate_pandas

0    1    2    3    4    5    6    7    8    9    ...  186  187  \
data x1 x2                                                    ...             
0    18 8     0    0    0    0    0    0    0    0    0    0  ...    0    0   
        9     0    0    0    0    0    0    0    0    0    0  ...    0    0   
        10    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        11    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        12    0    0    0    0    0    0    0    0    0    0  ...    0    0   
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
32   39 35    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        36    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        37    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        38    0    0    0    0    0    0    0    0    0    0  ...    0    0   
        39    0    0    0    0    0    0    0    0    0    0  ...    0    0   

            188  189  190  191  192  193  194  195  
data x1 x2                                          
0    18 8     0    0    0    0    0    0    0    0  
        9     0    0    0    0    0    0    0    0  
        10    0    0    0    0    0    0    0    0  
        11    0    0    0    0    0    0    0    0  
        12    0    0    0    0    0    0    0    0  
...         ...  ...  ...  ...  ...  ...  ...  ...  
32   39 35    0    0    0    0    0    0    0    0  
        36    0    0    0    0    0    0    0    0  
        37    0    0    0    0    0    0    0    0  
        38    0    0    0    0    0    0    0    0  
        39    0    0    0    0    0    0    0    0  

[22790 rows x 196 columns]

In [17]:
old_fktabledata.sigma

15            16            17            18   \
data x1 x2                                                           
0    18 8   7.886135e-11  2.789161e-12 -1.088094e-12 -4.117568e-13   
        9  -1.176740e-09 -3.219124e-11  1.584682e-11  5.996561e-12   
        10  1.914581e-08  5.518757e-10 -2.593601e-10 -9.814281e-11   
        11  3.395509e-08  2.585806e-09 -4.876171e-10 -1.843946e-10   
        12 -3.869336e-09  2.613134e-09 -1.856259e-10 -7.015371e-11   
...                  ...           ...           ...           ...   
32   39 35  1.334036e+02  2.590100e+02 -4.291108e-02 -8.200374e-02   
        36  1.276940e+02  2.480474e+02 -3.816512e-02 -7.178380e-02   
        37  1.224439e+02  2.379583e+02 -3.405945e-02 -6.363270e-02   
        38  1.177132e+02  2.288828e+02 -3.072791e-02 -5.726534e-02   
        39  1.132096e+02  2.202363e+02 -2.794119e-02 -5.141801e-02   

                     19            20            23            24   \
data x1 x2                                                           
0    18 8  -1.372518e-13  1.372526e-13  3.184748e-11  1.061583e-11   
        9   1.998853e-12 -1.998852e-12 -4.756445e-10 -1.585482e-10   
        10 -3.271441e-11  3.271440e-11  7.743511e-09  2.581170e-09   
        11 -6.146469e-11  6.146469e-11  1.368301e-08  4.561002e-09   
        12 -2.338454e-11  2.338465e-11 -1.557986e-09 -5.193287e-10   
...                  ...           ...           ...           ...   
32   39 35 -2.730750e-02  2.728248e-02  2.657959e-02  8.881789e-03   
        36 -2.390431e-02  2.388230e-02  2.332000e-02  7.792363e-03   
        37 -2.118991e-02  2.117047e-02  2.050479e-02  6.851812e-03   
        38 -1.906948e-02  1.905208e-02  1.829320e-02  6.112988e-03   
        39 -1.712237e-02  1.710665e-02  1.650276e-02  5.514523e-03   

                     25            29   ...           151           155  \
data x1 x2                              ...                               
0    18 8  -1.061583e-11  2.330620e-12  ...  3.532509e-12 -1.061748e-11   
        9   1.585482e-10 -3.473321e-11  ... -5.275854e-11  1.584912e-10   
        10 -2.581170e-09  5.643208e-10  ...  8.589165e-10 -2.580718e-09   
        11 -4.561002e-09  1.018451e-09  ...  1.517616e-09 -4.564180e-09   
        12  5.193284e-10 -8.114000e-11  ... -1.730282e-10  5.464645e-10   
...                  ...           ...  ...           ...           ...   
32   39 35 -8.827837e-03  2.892852e+01  ...  3.268192e-02  9.108678e+00   
        36 -7.745345e-03  2.769460e+01  ...  2.851808e-02  8.732327e+00   
        37 -6.810238e-03  2.655950e+01  ...  2.518946e-02  8.385249e+00   
        38 -6.075593e-03  2.553658e+01  ...  2.262572e-02  8.072070e+00   
        39 -5.481025e-03  2.456261e+01  ...  2.029938e-02  7.773046e+00   

                     156           157           158           159  \
data x1 x2                                                           
0    18 8  -3.746045e-13  1.370266e-13 -1.370346e-13 -4.567791e-14   
        9   4.328209e-12 -1.995623e-12  1.995686e-12  6.652257e-13   
        10 -7.431496e-11  3.266052e-11 -3.266170e-11 -1.088725e-11   
        11 -3.470094e-10  6.135241e-11 -6.136890e-11 -2.045638e-11   
        12 -3.454344e-10  2.329443e-11 -2.334610e-11 -7.782123e-12   
...                  ...           ...           ...           ...   
32   39 35  1.753977e+01 -3.793444e-02 -2.384179e-02  1.164286e-02   
        36  1.682565e+01 -3.362050e-02 -2.086757e-02  1.012728e-02   
        37  1.616641e+01 -3.011142e-02 -1.849971e-02  9.016706e-03   
        38  1.557250e+01 -2.729305e-02 -1.665334e-02  8.169412e-03   
        39  1.500474e+01 -2.471509e-02 -1.495479e-02  7.285986e-03   

                     160           163           164           165  
data x1 x2                                                          
0    18 8  -4.534985e-13  1.059753e-11  3.532509e-12  3.325722e-11  
        9   6.604470e-12 -1.582756e-10 -5.275854e-11 -4.965555e-10  
        10 -1.080917e-10  2.576749e-